In [1]:
pip install sklearn_crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 19.9 MB/s eta 0:00:00


In [2]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.2 MB/s eta 0:00:00


In [3]:
from sklearn_crfsuite import CRF
from sklearn.metrics import classification_report
import random
import itertools
import numpy as np
import scipy
from sklearn import metrics
from sklearn_crfsuite.utils import flatten
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

### Load Dataset

In [4]:
dataset = load_dataset("conll2003")
train_set = dataset["train"]
validation_set = dataset["validation"]
test_set = dataset["test"]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# A function to convert digit labels to string labels
def convert_ner_tags(dataset):
  # Label mapping from integer to string
  label_map = {0: 'O', 1: 'PER', 2: 'B-PER', 3: 'I-PER', 4: 'B-ORG', 5: 'I-ORG', 6: 'B-LOC', 7: 'I-LOC', 8: 'B-MISC', 9: 'I-MISC'}
  list_sent = []
  list_labels = []
  for sent in dataset:
    words = []
    label = []
    for word, label_idx in zip(sent['tokens'], sent['ner_tags']):
        label.append(label_map[label_idx])
        words.append(word)
    list_sent.append(words)
    list_labels.append(label)
  return list_sent, list_labels

In [6]:
# Define the train, validation and test set
train_ner, train_label_ner = convert_ner_tags(train_set)
validation_ner, validation_label_ner  = convert_ner_tags(validation_set)
test_ner, test_label_ner = convert_ner_tags(test_set)

In [7]:
print('Train',len(train_ner),len(train_label_ner))
print('Test',len(test_ner),len(test_label_ner))
print('Validation',len(validation_ner),len(validation_label_ner))

Train 14041 14041
Test 3453 3453
Validation 3250 3250


### Train CRF Model

In [8]:
# Initialize the CRF model with class weighting
crf = CRF(algorithm='lbfgs', c1=0, c2=1.0, max_iterations=300, all_possible_states=True)
crf.fit(train_ner, train_label_ner)
labels = list(crf.classes_)

### Evaluation

In [9]:
y_pred = crf.predict(test_ner)
print(test_label_ner[0])
print(y_pred[0])

['O', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'PER', 'O', 'O', 'O', 'O']
['I-PER', 'O', 'I-ORG', 'O', 'I-PER', 'B-ORG', 'O', 'I-ORG', 'O', 'I-ORG', 'O', 'O']


In [10]:
# Calculate evaluation metrics
report = classification_report(flatten(test_label_ner), flatten(y_pred), digits=3)
print(report)

              precision    recall  f1-score   support

       B-LOC      0.327     0.128     0.184       257
      B-MISC      0.165     0.167     0.166       216
       B-ORG      0.252     0.172     0.205       835
       B-PER      0.525     0.526     0.525      1156
       I-LOC      0.214     0.061     0.095       702
       I-ORG      0.410     0.328     0.364      1668
       I-PER      0.392     0.192     0.258      1661
           O      0.907     0.967     0.936     38323
         PER      0.515     0.381     0.438      1617

    accuracy                          0.848     46435
   macro avg      0.412     0.325     0.352     46435
weighted avg      0.819     0.848     0.830     46435

